# Non Interactive Zero Knowledge Proofs
#### by Jason Stillerman

In [2]:
import numpy as np
import random
import hashlib

coloring = {
    # outer five nodes, clockwise from top
    0: 'red',
    1: 'blue', 
    2: 'green',
    3: 'red',
    4: 'blue',
    # inner five nodes, clockwise from top
    5: 'blue',
    6: 'red',
    7: 'red',
    8: 'green',
    9: 'green'
}
print('Number of nodes:', len(coloring))

edges = [
    # outer shape, clockwise from top
    (0, 1),
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 0),
    # inner shape, clockwise from top
    (5, 0), (5, 7),
    (6, 1), (6, 8),
    (7, 2), (7, 9),
    (8, 3), (8, 5),
    (9, 4), (9, 6)
]
print('Number of edges:', len(edges))

Number of nodes: 10
Number of edges: 15


## Naive Solution

In [10]:
class NaiveProver:
    def shuffle_and_commit(self):
        colors = ['red', 'blue', 'green']
        shuffled_colors = colors.copy()
        random.shuffle(shuffled_colors)

        color_map = dict(zip(colors, shuffled_colors))

        self.shuffled_coloring = {}
        for node, color in coloring.items():
            self.shuffled_coloring[node] = color_map[color]

        self.committed_coloring_with_noise = {}
        committed_coloring = {}
        for node, color in self.shuffled_coloring.items():
            # create some noise so that the verifier can't tell if two nodes have the same color
            noise = ":" + str(random.random())

            # commit to the color and noise
            commitment = hashlib.sha256((color + noise).encode()).hexdigest()

            self.committed_coloring_with_noise[node] = color, noise
            committed_coloring[node] = commitment
            
        return (committed_coloring)
    
    def response(self, edge):
        # get the color + noise of each node in the edge
        color1, noise1 = self.committed_coloring_with_noise[edge[0]]
        color2, noise2 = self.committed_coloring_with_noise[edge[1]]
        
        c1 = (color1, noise1)
        c2 = (color2, noise2)
        
        # ask the verifier to check the response
        return (c1, c2)

    def create_example(self):
        commitment = self.shuffle_and_commit()
        random_oracle = int(hashlib.sha256(str(commitment).encode()).hexdigest(), 16)
        challenge_edge = edges[random_oracle % len(edges)]
        return (commitment, challenge_edge, self.response(challenge_edge))


class NaiveVerifier:

    def check(self, commitment, challenge_edge, response):
        c1, c2 = response

        color1, noise1 = c1
        color2, noise2 = c2

        # check that the challenge edge was picked fairly
        random_oracle = int(hashlib.sha256(str(commitment).encode()).hexdigest(), 16)
        real_challenge_edge = edges[random_oracle % len(edges)]

        assert challenge_edge == real_challenge_edge

        # check that the colors are different
        assert color1 != color2

        # check that the commitments are correct
        assert hashlib.sha256((color1 + noise1).encode()).hexdigest() == commitment[challenge_edge[0]]
        assert hashlib.sha256((color2 + noise2).encode()).hexdigest() == commitment[challenge_edge[1]]
        

def run_protocol():
    P = NaiveProver()
    V = NaiveVerifier()
    examples = [P.create_example() for _ in range(15**2)]
    for ex in examples:
        V.check(*ex)
    
run_protocol()

## Fixing the problem

This does not prevent the prover from re-picking their challenge until they get one that does not expose the cheating. In order to prevent the prover from cheating, they must commit to all recolorings up front and then use the random oracle hash function to generate a *sequence* of challenges using the entire library of commitments as the seed for the oracle.

## Secure Solution

In [15]:
class SecureProver:
    def shuffle_and_commit_single(self):
        colors = ['red', 'blue', 'green']
        shuffled_colors = colors.copy()
        random.shuffle(shuffled_colors)

        color_map = dict(zip(colors, shuffled_colors))

        self.shuffled_coloring = {}
        for node, color in coloring.items():
            self.shuffled_coloring[node] = color_map[color]

        committed_coloring_with_noise = {}
        committed_coloring = {}
        for node, color in self.shuffled_coloring.items():
            # create some noise so that the verifier can't tell if two nodes have the same color
            noise = ":" + str(random.random())

            # commit to the color and noise
            commitment = hashlib.sha256((color + noise).encode()).hexdigest()

            committed_coloring_with_noise[node] = color, noise
            committed_coloring[node] = commitment
            
        return (committed_coloring, committed_coloring_with_noise)

    def shuffle_and_commit(self, n):
        """n is the number of colorings we are committing to"""
        self.commitments = []
        self.commitments_with_noise = []

        for _ in range(n):
            # shuffle and commit to the coloring
            commitment, committed_coloring_with_noise = self.shuffle_and_commit_single()
            self.commitments.append(commitment)
            self.commitments_with_noise.append(committed_coloring_with_noise)

    def response(self, edge, committed_coloring_with_noise):
        # get the color + noise of each node in the edge
        color1, noise1 = committed_coloring_with_noise[edge[0]]
        color2, noise2 = committed_coloring_with_noise[edge[1]]
        
        c1 = (color1, noise1)
        c2 = (color2, noise2)
        
        # ask the verifier to check the response
        return (c1, c2)

    def create_examples(self):
        # start the random oracle as hash of all commitments
        random_oracle = int(hashlib.sha256(str(self.commitments).encode()).hexdigest(), 16)

        for commitment, committed_coloring_with_noise in zip(self.commitments, self.commitments_with_noise):
            # generate the next number in the random sequence by hashing the previous hash
            random_oracle = int(hashlib.sha256(str(random_oracle).encode()).hexdigest(), 16)
            challenge_edge = edges[random_oracle % len(edges)]
            yield (commitment, challenge_edge, self.response(challenge_edge, committed_coloring_with_noise))
        
class SecureVerifier:
    def check(self, examples):

        # initialize the random oracle with the same seed
        commitments = [commitment for commitment, _, _ in examples]
        random_oracle = int(hashlib.sha256(str(commitments).encode()).hexdigest(), 16)


        for commitment, challenge_edge, response in examples:
            
            c1, c2 = response

            color1, noise1 = c1
            color2, noise2 = c2

            # check that the challenge edge was picked fairly
            random_oracle = int(hashlib.sha256(str(random_oracle).encode()).hexdigest(), 16)
            real_challenge_edge = edges[random_oracle % len(edges)]

            assert challenge_edge == real_challenge_edge

            # check that the colors are different
            assert color1 != color2

            # check that the commitments are correct
            assert hashlib.sha256((color1 + noise1).encode()).hexdigest() == commitment[challenge_edge[0]]
            assert hashlib.sha256((color2 + noise2).encode()).hexdigest() == commitment[challenge_edge[1]]
            

def run_secure_protocol():
    P = SecureProver()
    V = SecureVerifier()

    P.shuffle_and_commit(15**2)
    V.check(P.create_examples())

run_secure_protocol()